In [31]:
import pandas as pd
import numpy as np
import os

In [32]:
# --- 1. CONFIGURACIÓN DE RUTAS Y CARGA ---
# Usamos 'os.path.join' para crear rutas que funcionen en cualquier sistema operativo.
INPUT_PATH = os.path.join('01_data', 'input', 'Online Retail.csv')
OUTPUT_PATH = os.path.join('01_data', 'output', 'master_df.csv')

print("Ruta de entrada:", INPUT_PATH)

try:
    # Carga del dataset con el encoding correcto
    initial_len = len(df)
    df = pd.read_csv(INPUT_PATH, encoding='ISO-8859-1')
    print("Dataset cargado exitosamente.")
except Exception as e:
    print(f"ERROR: No se pudo cargar el archivo. Verifica la ruta y el nombre (debe ser Online Retail.csv). Detalle: {e}")
    # Si hay error, detenemos la ejecución aquí.

Ruta de entrada: 01_data\input\Online Retail.csv
Dataset cargado exitosamente.


In [33]:
# 1. Manejo de Nulos: Eliminar registros sin ID de Cliente o Descripción.
df.dropna(subset=['CustomerID', 'Description'], inplace=True)

# 2. Conversión de Tipos de Datos (antes de la limpieza numérica)
df['CustomerID'] = df['CustomerID'].astype(int)
df['UnitPrice'] = df['UnitPrice'].astype(float)
df['Quantity'] = df['Quantity'].astype(int)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 3. Eliminar Duplicados (Esencial para transacciones)
df.drop_duplicates(inplace=True)


# --- 3. LIMPIEZA AVANZADA (Valores Atípicos y Errores) ---

# 4. Eliminar Precios y Cantidades Negativas o Cero (CRUCIAL para Elasticidad)
# Estas filas representan devoluciones, errores o productos gratuitos.
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

# 5. Eliminar Outliers Extremos de Precio
# Usamos el percentil 99.5% para eliminar precios inusualmente altos que distorsionan el modelo.
price_threshold = df['UnitPrice'].quantile(0.995)
df = df[df['UnitPrice'] < price_threshold]

# 6. Eliminar Outliers Extremos de Cantidad
# Usamos el percentil 99.5% para eliminar compras de inventario a granel o errores de ingreso.
quantity_threshold = df['Quantity'].quantile(0.995)
df = df[df['Quantity'] < quantity_threshold]

# 7. Limpieza por Tasa de Cancelación (Para identificar SKUs problemáticos)
# Eliminar códigos de stock que no son productos reales (ej. Códigos de ajuste 'POST', 'D', 'DOT')
non_product_codes = ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'CRUK']
df = df[~df['StockCode'].isin(non_product_codes)]

# 8. Filtrar por Duración del Dataset
# Nos enfocamos en el grueso del año de ventas, eliminando datos incompletos del inicio/final.
df = df[df['InvoiceDate'].dt.year == 2011]
# Filtraremos solo por las fechas más robustas (ej. Enero a Noviembre)
df = df[df['InvoiceDate'] < '2011-12-01'] 

print(f"Registros después de la limpieza avanzada: {len(df)}")
print(f"Total de registros eliminados: {initial_len - len(df)}")

Registros después de la limpieza avanzada: 344941
Total de registros eliminados: 0


In [34]:
# --- 3. FEATURE ENGINEERING Y RE-CONTEXTUALIZACIÓN GEOGRÁFICA Y DE MONEDA ---
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek # Lunes=0, Domingo=6

# Mapeo estratégico de países y Tasas de Conversión Simuladas
# Tasas simuladas: Multiplicador del precio original (GBP) para simular la moneda local.
conversion_rates = {
    'United Kingdom': {'new_country': 'MEXICO', 'rate': 25.0, 'currency': 'MXN'}, 
    'Germany': {'new_country': 'COLOMBIA', 'rate': 4500.0, 'currency': 'COP'},
    'France': {'new_country': 'CHILE', 'rate': 1000.0, 'currency': 'CLP'},
    'EIRE': {'new_country': 'PERU', 'rate': 4.0, 'currency': 'PEN'},
    'Spain': {'new_country': 'ARGENTINA', 'rate': 900.0, 'currency': 'ARS'},
    'Netherlands': {'new_country': 'BRASIL', 'rate': 6.0, 'currency': 'BRL'},
}

# Crear las nuevas columnas de País y Moneda
df['Original_Country'] = df['Country']
df['Country'] = df['Original_Country'].apply(
    lambda x: conversion_rates.get(x, {}).get('new_country', 'Otro_LATAM')
)
# ************** COLUMNA SOLICITADA **************
df['Local_Currency'] = df['Original_Country'].apply(
    lambda x: conversion_rates.get(x, {}).get('currency', 'USD')
)
# ************************************************

# Aplicar la conversión del precio a la moneda local simulada
def convert_price(row):
    original_country = row['Original_Country']
    rate = conversion_rates.get(original_country, {}).get('rate', 1.0)
    return row['UnitPrice'] * rate

df['UnitPrice_Local'] = df.apply(convert_price, axis=1)

print("Ajuste de moneda simulado completado.")



Ajuste de moneda simulado completado.


In [35]:
# --- 5. AGREGACIÓN DE DEMANDA Y SIMULACIÓN DE COMPETENCIA (CORREGIDA) ---

# A. Crear subconjunto de productos TOP
top_items = df.groupby('StockCode')['Quantity'].sum().nlargest(50).index
df_filtered = df[df['StockCode'].isin(top_items)]

# B. Agregación: Crearemos un master donde cada fila es una transición de cliente/producto/día.
# Esto asegura que el CustomerID se mantenga para el RFM.
# Usamos InvoiceNo en el groupby para mantener la unicidad de las transacciones (es un buen proxy).
df_master = df_filtered.groupby(
    ['InvoiceNo', 'StockCode', 'Country', 'Local_Currency', 'CustomerID', 'InvoiceDate', 'UnitPrice_Local']
).agg(
    # La cantidad total vendida en esa transacción específica
    Total_Quantity_Sold=('Quantity', 'sum')
).reset_index().rename(columns={'UnitPrice_Local': 'UnitPrice'})

# C. Crear Features de Tiempo (Se perdieron en la agregación, las re-creamos)
df_master['InvoiceDate'] = pd.to_datetime(df_master['InvoiceDate'])
df_master['Month'] = df_master['InvoiceDate'].dt.month
df_master['DayOfWeek'] = df_master['InvoiceDate'].dt.dayofweek

# D. Simulación del Precio de la Competencia (Competitor_Price)
df_master['Competitor_Price'] = np.random.normal(
    loc=df_master['UnitPrice'],
    scale=df_master['UnitPrice'] * 0.05
)
df_master['Competitor_Price'] = df_master['Competitor_Price'].apply(lambda x: max(x, 0.01))

print("\n✅ DataFrame Maestro regenerado con CustomerID a nivel de transacción.")

# --- 6. GUARDAR EL RESULTADO ---
# ASEGÚRATE DE GUARDAR EL ARCHIVO master_df.csv


✅ DataFrame Maestro regenerado con CustomerID a nivel de transacción.


In [36]:
# --- 5. GUARDAR EL RESULTADO ---
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df_master.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ FASE 1 COMPLETADA. Archivo master_df.csv guardado en: {OUTPUT_PATH}")


✅ FASE 1 COMPLETADA. Archivo master_df.csv guardado en: 01_data\output\master_df.csv
